In [2]:
!pip install pdfplumber openpyxl groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 942.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.3/137.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 76.3 MB/s eta 0:00:00


In [4]:
import pdfplumber
import openpyxl
import json
import re
import datetime
from groq import Groq
import pandas as pd
from google.colab import userdata

GROQ_API_KEY = userdata.get('GROQ_API_KEY')
client = Groq(api_key=GROQ_API_KEY)

In [5]:
def extract_text(pdf_path):
    #Extracting full text from the PDF
    full_text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                full_text += page_text + "\n"
    return full_text.strip()

In [6]:
def extract_structured_data(text):
    #Using Groq api to extract key-value pairs
    prompt = """
You are an expert data extraction AI specializing in converting unstructured biographical narratives into structured key-value pairs.

Given the following text:

{text}

Your task:
- Dynamically identify all factual elements and group them into logical key-value pairs. Keys should be concise and descriptive (e.g., "First Name", "Date of Birth", "Current Salary", "Certifications 1").
- For values: Use exact original data where possible.
  - Dates: Output in YYYY-MM-DD format if mentioned (or infer from context like "June 15, 2002" -> "15-Jun-02").
  - Salaries: Numeric value without commas or currency (e.g., "350,000 INR" -> 350000 for salary, separate "INR" as "Salary Currency").
  - Add Company/Organization name as value where there is salary mentioned, add previous, current prefix to the key value depending upon date of joining.
  - If no company name is mentioned (or just terms like "first company","last company",etc. is used), then keep the value section blank
  - Add date of joining and of leaving as and where is mentioned
  - Percentages/Scores: Keep as it is (e.g., "92.5%" -> 92.5%, "8.7 on a n-point scale" -> 8.7 and then add the scale in comment section).
  - Keep units in key or value if integral (e.g., "35 years" for age).
  - For lists like certifications or skills, create sequential keys (e.g., "Certifications 1", "Certifications 2").
  - For certifications or skills, add the certification exam/company in the value.
  - For certifications or skills, add the year of certification and marks in the comment section.
- For comments: Pull relevant contextual sentences or phrases from the original text using exact wording. Include all descriptive details, explanations, or additional info here. If a section is purely descriptive (e.g., technical skills paragraph), use an empty value and put the full description in comments.
- Ensure 100% capture: No summarization, omission, or paraphrasing unless absolutely needed for a clean key-value (e.g., inferring "Birth City" from "born in Jaipur"). Preserve original sentence structure in comments.
- Do not introduce new information.
- Output ONLY a valid JSON array of objects in this exact format: [{{"key": "string", "value": "string or number as string", "comments": "string"}}]
- Order logically: personal info, professional, education, certifications, skills.

Make the JSON parsable and complete.
"""


    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",  #LLM model used
        messages=[{"role": "user", "content": prompt.format(text=text)}],
        temperature=0.1,  #For low to no random generated text
        max_tokens=3000  #Max value being 7000, since using free tier
    )

    #Parsing the response as JSON
    try:
        json_str = response.choices[0].message.content.strip()
        if json_str.startswith('```json'):
            json_str = json_str[7:-3]  #For removing ```json and ```
        elif json_str.startswith('```'):
            json_str = json_str[3:-3]  #For removing ```
        data = json.loads(json_str)
        if not isinstance(data, list):
            raise ValueError("Not a list")
        return data
    except json.JSONDecodeError as e:
        raise ValueError(f"Failed to parse JSON from Groq response: {e}. Response: {response.choices[0].message.content}")

In [7]:
def parse_date(value):
    #Parsing a date string and return datetime.date, else returning original data.
    if not isinstance(value, str):
        return value
    value = value.strip()

    #List of possible date formats
    fmts = ["%d-%m-%Y", "%d/%m/%Y", "%Y-%m-%d", "%d-%b-%Y", "%d-%B-%Y"]

    for fmt in fmts:
        try:
            return datetime.datetime.strptime(value, fmt).date()
        except ValueError:
            pass

    return value

In [8]:
def create_excel_output(data, output_path):
    #Storing extracted data in excel file
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = "Output"

    ws.append(["Sr No.", "Key", "Value", "Comments"]) #Header row

    for row_idx, item in enumerate(data, start=2):
        ws.cell(row=row_idx, column=1, value=row_idx - 1)  # Sr no.
        ws.cell(row=row_idx, column=2, value=item.get("key", ""))

        raw_value = item.get("value", "")
        excel_value = parse_date(raw_value)
        cell = ws.cell(row=row_idx, column=3, value=excel_value)
        if isinstance(excel_value, datetime.date):
            cell.number_format = "DD-MMM-YY"

        ws.cell(row=row_idx, column=4, value=item.get("comments", ""))

    wb.save(output_path)
    print(f"Excel file saved to {output_path}")

In [9]:
def clean_comment(original_file):
  output_file   = '/content/Output.xlsx'

  df = pd.read_excel(original_file)
  comments = df['Comments'].fillna('').astype(str).str.strip()

  #Removing duplicates while keep the last occurrence of identical comments
  df['Comments'] = df['Comments'].where(~df['Comments'].duplicated(keep='last'), '')

  comments = df['Comments'].copy()
  for i in range(len(df)):
      if not comments[i]:
          continue
      current = comments[i].lower()
      for j in range(len(df)):
          if i != j and comments[j] and current in comments[j].lower():
              if len(comments[i]) <= len(comments[j]):  #Removing comments that are substrings of longer ones
                  df.at[i, 'Comments'] = ''
                  break

  from openpyxl import load_workbook

  wb = openpyxl.load_workbook(original_file)
  ws = wb.active

  #Writing only the comments column
  for row_idx, comment in enumerate(df['Comments'], start=2):
      ws.cell(row=row_idx, column=4).value = comment if comment else None

  wb.save(output_file)
  print("Cleaning completed")

In [11]:
if __name__ == "__main__":
    pdf_path = '/content/Data_Input.pdf'

    print("Extracting text from PDF...")
    text = extract_text(pdf_path)
    print("Text extracted successfully.")
    #print(f"Extracted text preview: {text[:500]}...") #For preview/debugging

    print("Extracting key-value pairs using Groq...")
    structured_data = extract_structured_data(text)
    print(f"Extracted {len(structured_data)} key-value pairs.")

    #For previewing/debugging key:value:comment
    #for item in structured_data:
        #print(f"Key: {item['key']}, Value: {item['value']}, Comments: {item['comments'][:100]}...")

    output_path = '/content/Output_1.xlsx'
    create_excel_output(structured_data, output_path)
    print("Excel file created successfully.")
    clean_comment(output_path)


Extracting text from PDF...
Text extracted successfully.
Extracting key-value pairs using Groq...
Extracted 44 key-value pairs.
Excel file saved to /content/Output_1.xlsx
Excel file created successfully.
Cleaning completed
